# Na aula anterior...

Qual o princípio base do SVM?

O SVM é um modelo linear?

# Lembrete

- Projeto

# Hoje!

**Para vocês**: Utilizando o dataset do titanic, realizar o seguinte processo:

1. Separar em dois dataframes de treino e teste.
1. Rodar um modelo simples de **SVM**, sem uso de pipeline, usando como resposta Survived e Pclass e Age como explicativa, substituindo missing pela mediana com uso de SimplesImputer;
2. Criar pipeline para rodar o modelo, adicionando apenas o SimpleImputer e o SVC - aqui, na hora do fit, passamos apenas as mesmas três colunas do step 2;
3. Usar o ColumnTransformer para criar a pipeline - manteremos o modelo usando apenas as mesmas duas colunas do step 1, com a diferença agora que o fit recebe todas as colunas e o **ColumnTransformer fará a seleção**;
4. Criar uma pipeline para dados numéricos (SimplesImputer com mediana, nas colunas Pclass, Age, SibSp e Parch) e uma outra pipeline para dados string (OrdinalEncoder, na coluna Sex, e SimpleImputer com a moda);
6. Adicionar uma **camada final para todas as variáveis** para fazer um redimensionamento com StandardScaler.
7. Refletir: podemos usar a coluna Name como variável explicativa para fazer previsão? É possível fazer algum pré-processamento nela?

In [85]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

from sklearn.svm import SVC

In [17]:
df = pd.read_csv("C:/Users/Avell/Google Drive/Luiz Gabriel - PC/06.Material de Cursos/LetsCode_DS_Curso/07_Machine_Learning_II/dados/titanic.csv")

### 1

In [20]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [21]:
var_expl = ["Pclass", "Age"]
y = "Survived"

In [24]:
df[var_expl].isnull().sum

Pclass      0
Age       177
dtype: int64

In [81]:
X_train, X_test, y_train, y_test = train_test_split(df[["Pclass", "Age"]],df["Survived"], test_size = 0.15, random_state = 1)

In [42]:
imputer = SimpleImputer(strategy='median')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

c:\Users\Avell\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


In [50]:
modelo = SVC(kernel = 'linear', probability =True)
modelo.fit(X_train,y_train)

SVC(kernel='linear', probability=True)

In [56]:
y_pred_train = modelo.predict_proba(X_train)[:, 1]
y_pred_test = modelo.predict_proba(X_test)[:, 1]

In [60]:
roc_treino = roc_auc_score(y_train, y_pred_train)
roc_teste = roc_auc_score(y_test, y_pred_test)

print("ROC-AUC Treino: ", roc_treino)
print("ROC-AUC Teste: ", roc_teste)

ROC-AUC Treino:  0.722673833214878
ROC-AUC Teste:  0.66875


### 2

In [82]:
modelo = SVC(
    kernel = 'linear',
    probability =True
)


pipe_imputer = Pipeline(steps = [
    ('pipe_imputer', SimpleImputer(strategy="median"))
])

pipe_transform = ColumnTransformer(transformers = [
    ('pipe_imputer', pipe_imputer, ["Pclass", "Age"])
])

pipeline = Pipeline(steps = [
    ('preproc', pipe_transform),
    ('modelo', modelo)
])

pipeline.fit(X_train, y_train)

y_pred_trein = pipeline.predict_proba(X_train)[:, 1]
y_pred_test = pipeline.predict_proba(X_test)[:, 1]

roc_treino = roc_auc_score(y_train, y_pred_train)
roc_teste = roc_auc_score(y_test, y_pred_test)

print("ROC-AUC Treino: ", roc_treino)
print("ROC-AUC Teste: ", roc_teste)

ROC-AUC Treino:  0.722673833214878
ROC-AUC Teste:  0.6659722222222222


### 3

In [83]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns = "Survived"), df["Survived"], test_size = 0.15, random_state = 1)

In [129]:
modelo = SVC(
    kernel = 'linear',
    probability =True
)

lista_ordenada_sex = ['female', 'male']


pipe_imputer_num = Pipeline(steps = [
    ('pipe_imputer', SimpleImputer(strategy="median"))
])

pipe_imputer_cat = Pipeline(steps = [
    ('pipe_imputer', SimpleImputer(strategy="most_frequent"))
])

pipe_encoder = Pipeline(steps = [
    ('pipe_imputer', OrdinalEncoder(categories=[lista_ordenada_sex]))
])

pipe_columns = ColumnTransformer(transformers = [
    ('pipe_imputer_nums', pipe_imputer_num, ["Pclass", "Age", "SibSp", "Parch"]),
    ('pipe_imputer_cat', pipe_imputer_cat, ["Sex"]),    
    ('pipe_encoder', pipe_encoder, ["Sex"])
])

pipe_std = Pipeline(steps = [
    ('pipe_std', StandardScaler())
])


pipeline = Pipeline(steps = [
    ('preproc', pipe_columns),
    ('pipe_std', pipe_std),
    ('modelo', modelo)
])

pipeline.fit(X_train, y_train)

y_pred_trein = pipeline.predict_proba(X_train)[:, 1]
y_pred_test = pipeline.predict_proba(X_test)[:, 1]

roc_treino = roc_auc_score(y_train, y_pred_train)
roc_teste = roc_auc_score(y_test, y_pred_test)

print("ROC-AUC Treino: ", roc_treino)
print("ROC-AUC Teste: ", roc_teste)

ValueError: could not convert string to float: 'female'

In [119]:
pipe_tranforma = Pipeline(steps = pipeline.steps[:-1])
pipe_tranforma.transform(X_train)

array([[ 0.82594713,  0.25548285,  2.43771768, -0.47848839, -1.34689509],
       [-1.56503557,  2.13675226, -0.49296355, -0.47848839, -1.34689509],
       [-1.56503557, -0.8732788 , -0.49296355,  2.05448861, -1.34689509],
       ...,
       [-0.36954422, -0.64752647, -0.49296355, -0.47848839,  0.74244832],
       [ 0.82594713, -0.08314565, -0.49296355, -0.47848839, -1.34689509],
       [ 0.82594713, -0.64752647, -0.49296355, -0.47848839,  0.74244832]])

In [120]:
from copy import deepcopy

passos = deepcopy(pipeline.steps[:-1])


In [122]:
passos.insert(1, "ABC")

In [123]:
passos

[('preproc',
  ColumnTransformer(transformers=[('pipe_imputer',
                                   Pipeline(steps=[('pipe_imputer',
                                                    SimpleImputer(strategy='median'))]),
                                   ['Pclass', 'Age', 'SibSp', 'Parch']),
                                  ('pipe_encoder',
                                   Pipeline(steps=[('pipe_imputer',
                                                    OrdinalEncoder(categories=[['female',
                                                                                'male']]))]),
                                   ['Sex'])])),
 'ABC',
 ('pipe_std', Pipeline(steps=[('pipe_std', StandardScaler())]))]

In [ ]:
modelo = SVC(
    kernel = 'linear',
    probability =True
)

lista_ordenada_sex = ['female', 'male']

pipe_imputer = Pipeline(steps = [
    ('pipe_imputer', SimpleImputer(strategy="median"))
])

pipe_encoder = Pipeline(steps = [
    ('pipe_imputer', OrdinalEncoder(categories=[lista_ordenada_sex]))
])

pipe_columns = ColumnTransformer(transformers = [
    ('pipe_imputer', pipe_imputer, ["Pclass", "Age", "SibSp", "Parch"]),
    ('pipe_encoder', pipe_encoder, ["Sex"])
])

pipe_std = Pipeline(steps = [
    ('pipe_std', StandardScaler())
])


pipeline = Pipeline(steps = [
    ('preproc', pipe_columns),
    ('pipe_std', pipe_std)
])

X = pipeline.fit_transform(X_train)

In [107]:
pipeline.get_params

<bound method Pipeline.get_params of Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('pipe_imputer',
                                                  Pipeline(steps=[('pipe_imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch']),
                                                 ('pipe_encoder',
                                                  Pipeline(steps=[('pipe_imputer',
                                                                   OrdinalEncoder(categories=[['female',
                                                                                               'male']]))]),
                                                  ['Sex'])])),
                ('pipe_std', Pipeline(steps=[('pipe_std', StandardScaler())]))])>

## Algoritmos de Clusterização

### K-Means

1. Escolha aleatória de N exemplos do conjunto de dados que serão os centróides iniciais;
2. Por meio de uma métrica de proximidade, assimila cada instância do conjunto de dados a um centróide;
3. Cria novos N centróides baseado na média de todos os exemplos associados a um centróide específico.

<img src="imagens/cluster_kmean.png"/>

### DBSCAN

1. Dado um ponto p, encontra todos os pontos q que estão a uma distância igual ou inferior a $\varepsilon$ de p;
2. Repete o processo para todos os pontos encontrados, e para todos os pontos novos, recursivamente, determinando assim um grupo;
3. Procura um ponto ainda não visitado para iniciar um novo grupo e repetir os passos 1 e 2.
4. Determina como ruído os pontos que não fazem parte de nenhum grupo.

<img src="imagens/cluster_dbscan.png"/>